In [1]:
import pandas as pd
import numpy as np

In [23]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

In [2]:
LOCI = ['D3S1358_1', 'D3S1358_2',
       'D1S1656_1', 'D1S1656_2', 'D2S441_1', 'D2S441_2', 'D10S1248_1',
       'D10S1248_2', 'D13S317_1', 'D13S317_2', 'Penta_E_1', 'Penta_E_2',
       'D16S539_1', 'D16S539_2', 'D18S51_1', 'D18S51_2', 'D2S1338_1',
       'D2S1338_2', 'CSF1PO_1', 'CSF1PO_2', 'Penta_D_1', 'Penta_D_2', 'TH01_1',
       'TH01_2', 'vWA_1', 'vWA_2', 'D21S11_1', 'D21S11_2', 'D7S820_1',
       'D7S820_2', 'D5S818_1', 'D5S818_2', 'TPOX_1', 'TPOX_2', 'D8S1179_1',
       'D8S1179_2', 'D12S391_1', 'D12S391_2', 'D19S433_1', 'D19S433_2',
       'SE33_1', 'SE33_2', 'D22S1045_1', 'D22S1045_2', 'FGA_1', 'FGA_2',
       'D9S1122_1', 'D9S1122_2', 'D20S482_1', 'D20S482_2', 'D6S1043_1',
       'D6S1043_2', 'D17S1301_1', 'D17S1301_2', 'D4S2408_1', 'D4S2408_2']
TARGET = 'Ethno_hist_region'

In [122]:
train_df = pd.read_csv('train_data.csv')
test_df = pd.read_csv('test_data.csv')


def fix_columns(df):
    for locus in LOCI:
        if df[locus].dtype != object:
            continue
        print(locus, 'fixed')
        df[locus] = df[locus].apply(lambda x: float(x) if x[-1]!= '.' else float(x[:-1]))
    return df
    
train_df = fix_columns(train_df)
test_df = fix_columns(test_df)

train_df.shape, test_df.shape

D19S433_1 fixed


((448, 60), (100, 60))

# Feature selection

**Univariance**

In [108]:
X, y = train_df[LOCI], train_df[TARGET]

In [109]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
X_new_chi2 = SelectKBest(chi2, k=20).fit_transform(X, y)
X_new_f_classif = SelectKBest(f_classif, k=20).fit_transform(X, y)
X_new_mutual_info_classif = SelectKBest(mutual_info_classif, k=20).fit_transform(X, y)

In [110]:
def get_selected_columns(old_df, new_df):
    columns = []
    for i in range(new_df.shape[1]):
        for column in old_df.columns:
            if (old_df[column] == new_df[:, i]).all():
                columns.append(column)
                break
    return columns

chi2_columns = get_selected_columns(train_df, X_new_chi2)
f_classif_columns = get_selected_columns(train_df, X_new_f_classif)
mutual_info_columns = get_selected_columns(train_df, X_new_mutual_info_classif)

In [111]:
(
    set(chi2_columns) 
    set(f_classif_columns) &
    set(mutual_info_columns)
)

{'D12S391_1',
 'D12S391_2',
 'D13S317_1',
 'D18S51_1',
 'D18S51_2',
 'D1S1656_1',
 'D1S1656_2',
 'D22S1045_1',
 'D2S1338_1',
 'D2S1338_2',
 'D6S1043_1',
 'D6S1043_2',
 'D7S820_1',
 'Penta_D_1',
 'Penta_E_1',
 'Penta_E_2',
 'SE33_1',
 'SE33_2',
 'TH01_2',
 'TPOX_2'}

**RFE**

In [52]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

# Create the RFE object and compute a cross-validated score.
svc = SVC(kernel="linear")
# The "accuracy" scoring is proportional to the number of correct
# classifications
rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(5),
              scoring='accuracy')
rfecv.fit(X, y)

RFECV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
      estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                    decision_function_shape='ovr', degree=3,
                    gamma='auto_deprecated', kernel='linear', max_iter=-1,
                    probability=False, random_state=None, shrinking=True,
                    tol=0.001, verbose=False),
      min_features_to_select=1, n_jobs=None, scoring='accuracy', step=1,
      verbose=0)

In [61]:
sorted(list(zip(LOCI, rfecv.ranking_)), key=lambda x: x[1])

[('TH01_2', 1),
 ('D7S820_1', 2),
 ('D9S1122_1', 3),
 ('D10S1248_1', 4),
 ('CSF1PO_2', 5),
 ('D3S1358_2', 6),
 ('CSF1PO_1', 7),
 ('D21S11_2', 8),
 ('D7S820_2', 9),
 ('D17S1301_1', 10),
 ('D1S1656_2', 11),
 ('D4S2408_1', 12),
 ('D4S2408_2', 13),
 ('D9S1122_2', 14),
 ('D19S433_2', 15),
 ('D19S433_1', 16),
 ('D10S1248_2', 17),
 ('D5S818_1', 18),
 ('D5S818_2', 19),
 ('TPOX_2', 20),
 ('D12S391_1', 21),
 ('D8S1179_2', 22),
 ('D13S317_2', 23),
 ('D20S482_1', 24),
 ('TPOX_1', 25),
 ('D12S391_2', 26),
 ('D3S1358_1', 27),
 ('D20S482_2', 28),
 ('D16S539_2', 29),
 ('D16S539_1', 30),
 ('D17S1301_2', 31),
 ('vWA_2', 32),
 ('D13S317_1', 33),
 ('Penta_D_1', 34),
 ('Penta_D_2', 35),
 ('D18S51_2', 36),
 ('D18S51_1', 37),
 ('TH01_1', 38),
 ('D8S1179_1', 39),
 ('D6S1043_1', 40),
 ('D1S1656_1', 41),
 ('D22S1045_2', 42),
 ('FGA_1', 43),
 ('D21S11_1', 44),
 ('D2S1338_2', 45),
 ('Penta_E_1', 46),
 ('D2S441_1', 47),
 ('vWA_1', 48),
 ('FGA_2', 49),
 ('D2S441_2', 50),
 ('D2S1338_1', 51),
 ('D22S1045_1', 52),
 ('

**L1 based + linear model**

In [115]:



lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_l1 = model.transform(X)
X_l1.shape

/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


(448, 17)

In [116]:
get_selected_columns(train_df, X_l1)

['D1S1656_2',
 'D10S1248_2',
 'Penta_E_1',
 'Penta_E_2',
 'D18S51_1',
 'D18S51_2',
 'D2S1338_1',
 'D2S1338_2',
 'D21S11_2',
 'D7S820_1',
 'TPOX_2',
 'D12S391_1',
 'D12S391_2',
 'SE33_1',
 'SE33_2',
 'D6S1043_1',
 'D6S1043_2']

**Tree based**

In [113]:

clf = ExtraTreesClassifier(n_estimators=100)
clf = clf.fit(X, y)
clf.feature_importances_  
model = SelectFromModel(clf, prefit=True)
X_tree = model.transform(X)
X.shape, X_tree.shape

((448, 56), (448, 31))

In [118]:
q = set(get_selected_columns(train_df, X_tree)) & set(get_selected_columns(train_df, X_l1))

In [120]:
w = [('TH01_2', 1),
 ('D7S820_1', 2),
 ('D9S1122_1', 3),
 ('D10S1248_1', 4),
 ('CSF1PO_2', 5),
 ('D3S1358_2', 6),
 ('CSF1PO_1', 7),
 ('D21S11_2', 8),
 ('D7S820_2', 9),
 ('D17S1301_1', 10),
 ('D1S1656_2', 11),
 ('D4S2408_1', 12),
 ('D4S2408_2', 13),
 ('D9S1122_2', 14),
 ('D19S433_2', 15),
 ('D19S433_1', 16),
 ('D10S1248_2', 17),
 ('D5S818_1', 18),
 ('D5S818_2', 19),
 ('TPOX_2', 20),
 ('D12S391_1', 21),
 ('D8S1179_2', 22),
 ('D13S317_2', 23),
 ('D20S482_1', 24),
 ('TPOX_1', 25),
 ('D12S391_2', 26),
 ('D3S1358_1', 27),
 ('D20S482_2', 28),
 ('D16S539_2', 29),
 ('D16S539_1', 30),
 ('D17S1301_2', 31),
 ('vWA_2', 32),
 ('D13S317_1', 33),
 ('Penta_D_1', 34),
 ('Penta_D_2', 35),
 ('D18S51_2', 36),
 ('D18S51_1', 37),
 ('TH01_1', 38),
 ('D8S1179_1', 39),
 ('D6S1043_1', 40),
 ('D1S1656_1', 41),
 ('D22S1045_2', 42),
 ('FGA_1', 43),
 ('D21S11_1', 44),
 ('D2S1338_2', 45),
 ('Penta_E_1', 46),
 ('D2S441_1', 47),
 ('vWA_1', 48),
 ('FGA_2', 49),
 ('D2S441_2', 50),
 ('D2S1338_1', 51),
 ('D22S1045_1', 52),
 ('Penta_E_2', 53),
 ('SE33_1', 54),
 ('D6S1043_2', 55),
 ('SE33_2', 56)]

In [121]:
for x, y in w:
    if x in q:
        print(x)

D7S820_1
D21S11_2
D1S1656_2
D10S1248_2
TPOX_2
D12S391_1
D12S391_2
D18S51_2
D18S51_1
D2S1338_2
Penta_E_1
D2S1338_1
Penta_E_2
SE33_1
D6S1043_2
SE33_2


In [ ]:
clf = Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(penalty="l1"))),
  ('classification', RandomForestClassifier())
])
clf.fit(X, y)


# Classificaton

**train/val split**

In [131]:
from sklearn.model_selection import train_test_split

index = train_df.index

train_index, valid_index = train_test_split(index, stratify = train_df.Ethno_hist_region, shuffle=True, test_size=60)

X_train, X_valid = train_df.iloc[train_index], train_df.iloc[valid_index]
X_train.shape, X_valid.shape

((388, 60), (60, 60))

**super base line**

In [132]:
print("TRAIN: \n", "ACC: \n", np.mean('Центральный' == X_train[TARGET]))
print("VALID: \n", "ACC: \n", np.mean('Центральный' == X_valid[TARGET]))

TRAIN: 
 ACC: 
 0.3118556701030928
VALID: 
 ACC: 
 0.31666666666666665


**baseline**

In [133]:


baseline_lgbm_model = LGBMClassifier(num_leaves=6).fit(
    X_train[LOCI], X_train[TARGET],
    feature_name=LOCI,
    eval_set=(X_valid[LOCI], X_valid[TARGET]),
    verbose=False
)

In [134]:
def report_acc(model, X_train):
    
    try:
        print(sorted(list(zip(LOCI, model.feature_importances_)), key=lambda x: -x[1])[:10])
    except:
        pass
    
    y_pred = model.predict(X_train[LOCI])
    y_true = X_train[TARGET]
    print("TRAIN: \n", "ACC: \n", np.mean(y_pred == y_true), "\n", classification_report(y_true, y_pred))
    
    y_pred = model.predict(X_valid[LOCI])
    y_true = X_valid[TARGET]
    print("VALID: \n", "ACC: \n", np.mean(y_pred == y_true), "\n", classification_report(y_true, y_pred))
    
    print(pd.Series(model.predict(X_valid[LOCI])).value_counts())

In [135]:
report_acc(baseline_lgbm_model, X_train)

[('SE33_2', 116), ('D12S391_1', 107), ('SE33_1', 103), ('D7S820_1', 96), ('D2S1338_1', 88), ('D1S1656_2', 87), ('D8S1179_1', 85), ('D21S11_2', 84), ('D12S391_2', 81), ('TPOX_2', 74)]
TRAIN: 
 ACC: 
 0.9948453608247423 
                precision    recall  f1-score   support

Вост. Полесье       1.00      1.00      1.00        39
 Зап. Полесье       1.00      0.98      0.99        51
  Поднепровье       1.00      0.99      0.99        76
    Понеманье       1.00      1.00      1.00        57
     Поозерье       1.00      1.00      1.00        44
  Центральный       0.98      1.00      0.99       121

     accuracy                           0.99       388
    macro avg       1.00      0.99      1.00       388
 weighted avg       0.99      0.99      0.99       388

VALID: 
 ACC: 
 0.25 
                precision    recall  f1-score   support

Вост. Полесье       0.20      0.17      0.18         6
 Зап. Полесье       0.17      0.12      0.14         8
  Поднепровье       0.20      0.18    

## Other models

**NearestNeighbors**

In [136]:
%%time 


nn_params = {
    'classification__n_neighbors': [1, 5, 10],
    'classification__weights': ['uniform', 'distance'],
    'classification__leaf_size': [1, 5, 10],
    'classification__p': [1, 2],
}

nn_pipe = Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(C=0.01, penalty="l1", dual=False))),
  ('normalization', StandardScaler()),
  ('classification', KNeighborsClassifier())
])
nn_clf = GridSearchCV(nn_pipe, nn_params, cv=4, n_jobs=-1, scoring='f1_macro')
nn_clf.fit(X_train[LOCI], X_train[TARGET])

CPU times: user 1.01 s, sys: 82.8 ms, total: 1.1 s
Wall time: 21 s


/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


GridSearchCV(cv=4, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('feature_selection',
                                        SelectFromModel(estimator=LinearSVC(C=0.01,
                                                                            class_weight=None,
                                                                            dual=False,
                                                                            fit_intercept=True,
                                                                            intercept_scaling=1,
                                                                            loss='squared_hinge',
                                                                            max_iter=1000,
                                                                            multi_class='ovr',
                                                                            penalty='l1',
                        

In [138]:
report_acc(nn_clf.best_estimator_, X_train)

TRAIN: 
 ACC: 
 1.0 
                precision    recall  f1-score   support

Вост. Полесье       1.00      1.00      1.00        39
 Зап. Полесье       1.00      1.00      1.00        51
  Поднепровье       1.00      1.00      1.00        76
    Понеманье       1.00      1.00      1.00        57
     Поозерье       1.00      1.00      1.00        44
  Центральный       1.00      1.00      1.00       121

     accuracy                           1.00       388
    macro avg       1.00      1.00      1.00       388
 weighted avg       1.00      1.00      1.00       388

VALID: 
 ACC: 
 0.2 
                precision    recall  f1-score   support

Вост. Полесье       0.12      0.17      0.14         6
 Зап. Полесье       0.00      0.00      0.00         8
  Поднепровье       0.15      0.18      0.17        11
    Понеманье       0.25      0.33      0.29         9
     Поозерье       0.14      0.14      0.14         7
  Центральный       0.38      0.26      0.31        19

     accuracy   

**Naive Bayes**

In [139]:
%%time


nb_params = {
    'classification__alpha': np.linspace(0, 10, 10),
}

nb_pipe = Pipeline([
  #('feature_selection', SelectFromModel(LinearSVC(C=0.01, penalty="l1", dual=False))),
  #('normalization', StandardScaler()),
  ('classification', MultinomialNB())
])
nb_clf = GridSearchCV(nb_pipe, nb_params, cv=4, n_jobs=-1, scoring='f1_macro')
nb_clf.fit(X_train[LOCI], X_train[TARGET])

CPU times: user 154 ms, sys: 9.06 ms, total: 163 ms
Wall time: 408 ms


/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


GridSearchCV(cv=4, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('classification',
                                        MultinomialNB(alpha=1.0,
                                                      class_prior=None,
                                                      fit_prior=True))],
                                verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'classification__alpha': array([ 0.        ,  1.11111111,  2.22222222,  3.33333333,  4.44444444,
        5.55555556,  6.66666667,  7.77777778,  8.88888889, 10.        ])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_macro', verbose=0)

In [140]:
report_acc(nb_clf.best_estimator_, X_train)

TRAIN: 
 ACC: 
 0.33505154639175255 
                precision    recall  f1-score   support

Вост. Полесье       0.00      0.00      0.00        39
 Зап. Полесье       0.39      0.22      0.28        51
  Поднепровье       0.43      0.04      0.07        76
    Понеманье       0.00      0.00      0.00        57
     Поозерье       0.00      0.00      0.00        44
  Центральный       0.33      0.96      0.49       121

     accuracy                           0.34       388
    macro avg       0.19      0.20      0.14       388
 weighted avg       0.24      0.34      0.20       388

VALID: 
 ACC: 
 0.2833333333333333 
                precision    recall  f1-score   support

Вост. Полесье       0.00      0.00      0.00         6
 Зап. Полесье       0.14      0.12      0.13         8
  Поднепровье       0.00      0.00      0.00        11
    Понеманье       0.00      0.00      0.00         9
     Поозерье       0.00      0.00      0.00         7
  Центральный       0.30      0.84      0

/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Decision tree**

In [141]:
%%time

dt_params = {
    'classification__criterion': ['gini', 'entropy'],
    'classification__max_depth': [1, 3, 5, 10],
    'classification__class_weight': [None, 'balanced']
}

dt_pipe = Pipeline([
  #('feature_selection', SelectFromModel(LinearSVC(C=0.01, penalty="l1", dual=False))),
  #('normalization', StandardScaler()),
  ('classification', DecisionTreeClassifier())
])
dt_clf = GridSearchCV(nb_pipe, nb_params, cv=4, n_jobs=-1, scoring='f1_macro')
dt_clf.fit(X_train[LOCI], X_train[TARGET])

CPU times: user 281 ms, sys: 61.7 ms, total: 343 ms
Wall time: 10.2 s


/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


GridSearchCV(cv=4, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('classification',
                                        MultinomialNB(alpha=1.0,
                                                      class_prior=None,
                                                      fit_prior=True))],
                                verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'classification__alpha': array([ 0.        ,  1.11111111,  2.22222222,  3.33333333,  4.44444444,
        5.55555556,  6.66666667,  7.77777778,  8.88888889, 10.        ])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_macro', verbose=0)

In [157]:
report_acc(dt_clf.best_estimator_, X_train)

TRAIN: 
 ACC: 
 0.33505154639175255 
                precision    recall  f1-score   support

Вост. Полесье       0.00      0.00      0.00        39
 Зап. Полесье       0.39      0.22      0.28        51
  Поднепровье       0.43      0.04      0.07        76
    Понеманье       0.00      0.00      0.00        57
     Поозерье       0.00      0.00      0.00        44
  Центральный       0.33      0.96      0.49       121

     accuracy                           0.34       388
    macro avg       0.19      0.20      0.14       388
 weighted avg       0.24      0.34      0.20       388

VALID: 
 ACC: 
 0.2833333333333333 
                precision    recall  f1-score   support

Вост. Полесье       0.00      0.00      0.00         6
 Зап. Полесье       0.14      0.12      0.13         8
  Поднепровье       0.00      0.00      0.00        11
    Понеманье       0.00      0.00      0.00         9
     Поозерье       0.00      0.00      0.00         7
  Центральный       0.30      0.84      0

/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Random forest**

In [174]:
%%time


rf_params = {
    'classification__max_leaf_nodes': [1, 3, 16, 32, 64, 200],
    'classification__class_weight': [None, 'balanced']
}

rf_pipe = Pipeline([
  #('feature_selection', SelectFromModel(LinearSVC(C=0.01, penalty="l1", dual=False))),
  #('normalization', StandardScaler()),
  ('classification', RandomForestClassifier())
])
rf_clf = GridSearchCV(nb_pipe, nb_params, cv=4, n_jobs=-1, scoring='f1_macro')
rf_clf.fit(X_train[LOCI], X_train[TARGET])

CPU times: user 121 ms, sys: 9.84 ms, total: 131 ms
Wall time: 339 ms


/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


GridSearchCV(cv=4, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('classification',
                                        MultinomialNB(alpha=1.0,
                                                      class_prior=None,
                                                      fit_prior=True))],
                                verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'classification__alpha': array([ 0.        ,  1.11111111,  2.22222222,  3.33333333,  4.44444444,
        5.55555556,  6.66666667,  7.77777778,  8.88888889, 10.        ])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_macro', verbose=0)

In [175]:
report_acc(rf_clf.best_estimator_, X_train)

TRAIN: 
 ACC: 
 0.33505154639175255 
                precision    recall  f1-score   support

Вост. Полесье       0.00      0.00      0.00        39
 Зап. Полесье       0.39      0.22      0.28        51
  Поднепровье       0.43      0.04      0.07        76
    Понеманье       0.00      0.00      0.00        57
     Поозерье       0.00      0.00      0.00        44
  Центральный       0.33      0.96      0.49       121

     accuracy                           0.34       388
    macro avg       0.19      0.20      0.14       388
 weighted avg       0.24      0.34      0.20       388

VALID: 
 ACC: 
 0.2833333333333333 
                precision    recall  f1-score   support

Вост. Полесье       0.00      0.00      0.00         6
 Зап. Полесье       0.14      0.12      0.13         8
  Поднепровье       0.00      0.00      0.00        11
    Понеманье       0.00      0.00      0.00         9
     Поозерье       0.00      0.00      0.00         7
  Центральный       0.30      0.84      0

/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**svm**

In [28]:
%%time

svc_params = {
    'classification__penalty': ['l1', 'l2'],
    'classification__tol': [1e-4],
    'classification__C': [1e-1, 1e-2, 1e1, 1, 1e2],
}

svc_pipe = Pipeline([
  #('feature_selection', SelectFromModel(LinearSVC(C=0.01, penalty="l1", dual=False))),
  #('normalization', StandardScaler()),
  ('classification', LinearSVC())
])
svc_clf = GridSearchCV(svc_pipe, svc_params, cv=3, n_jobs=-1, scoring='f1_macro')
svc_clf.fit(X_train[LOCI], X_train[TARGET])

CPU times: user 524 ms, sys: 7.1 ms, total: 531 ms
Wall time: 4.14 s


/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('classification',
                                        LinearSVC(C=1.0, class_weight=None,
                                                  dual=True, fit_intercept=True,
                                                  intercept_scaling=1,
                                                  loss='squared_hinge',
                                                  max_iter=1000,
                                                  multi_class='ovr',
                                                  penalty='l2',
                                                  random_state=None, tol=0.0001,
                                                  verbose=0))],
                                verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'classification__C': [0.1, 0.01, 10.0, 1, 100.0],
                         'classification__penalty': [

In [29]:
report_acc(svc_clf.best_estimator_)

TRAIN: 
 ACC: 
 0.48711340206185566 
                precision    recall  f1-score   support

Вост. Полесье       0.35      0.15      0.21        39
 Зап. Полесье       0.55      0.51      0.53        51
  Поднепровье       0.49      0.43      0.46        76
    Понеманье       0.53      0.28      0.37        57
     Поозерье       0.58      0.16      0.25        44
  Центральный       0.47      0.83      0.60       121

     accuracy                           0.49       388
    macro avg       0.50      0.40      0.40       388
 weighted avg       0.49      0.49      0.45       388

VALID: 
 ACC: 
 0.25 
                precision    recall  f1-score   support

Вост. Полесье       0.00      0.00      0.00         6
 Зап. Полесье       0.25      0.25      0.25         8
  Поднепровье       0.17      0.09      0.12        11
    Понеманье       0.33      0.11      0.17         9
     Поозерье       0.00      0.00      0.00         7
  Центральный       0.28      0.58      0.38        19


# Over sampling

**random**

In [158]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, _ = ros.fit_resample(X_train, X_train[TARGET])

In [159]:
X_resampled[TARGET].value_counts()

Зап. Полесье     121
Понеманье        121
Центральный      121
Поозерье         121
Поднепровье      121
Вост. Полесье    121
Name: Ethno_hist_region, dtype: int64

In [160]:


def check_oversampling(new_X_train):

    baseline_lgbm_model = LGBMClassifier(num_leaves=16).fit(
        new_X_train[LOCI], new_X_train[TARGET],
        feature_name=LOCI,
        eval_set=(X_valid[LOCI], X_valid[TARGET]),
    )
    print("!!!!!!LGBM!!!!!!!")
    report_acc(baseline_lgbm_model, new_X_train)

    nn_params = {
        'classification__n_neighbors': [1, 5, 10],
        'classification__weights': ['uniform', 'distance'],
        'classification__leaf_size': [1, 5, 10],
        'classification__p': [1, 2],
    }

    nn_pipe = Pipeline([
      ('feature_selection', SelectFromModel(LinearSVC(C=0.01, penalty="l1", dual=False))),
      ('normalization', StandardScaler()),
      ('classification', KNeighborsClassifier())
    ])
    nn_clf = GridSearchCV(nn_pipe, nn_params, cv=4, n_jobs=-1, scoring='f1_macro')
    nn_clf.fit(new_X_train[LOCI], new_X_train[TARGET])
    
    print("!!!!!!NN!!!!!!!")
    report_acc(nn_clf.best_estimator_, new_X_train)
    

    nb_params = {
        'classification__alpha': np.linspace(0, 10, 10),
    }

    nb_pipe = Pipeline([
        #('feature_selection', SelectFromModel(LinearSVC(C=0.01, penalty="l1", dual=False))),
        #('normalization', StandardScaler()),
        ('classification', MultinomialNB())
    ])
    nb_clf = GridSearchCV(nb_pipe, nb_params, cv=4, n_jobs=-1, scoring='f1_macro')
    nb_clf.fit(new_X_train[LOCI], new_X_train[TARGET])
    
    print("!!!!!!NB!!!!!!!")
    report_acc(nb_clf.best_estimator_, new_X_train)
    
    
    dt_params = {
        'classification__criterion': ['gini', 'entropy'],
        'classification__max_depth': [1, 3, 5, 10],
        'classification__class_weight': [None, 'balanced']
    }

    dt_pipe = Pipeline([
      #('feature_selection', SelectFromModel(LinearSVC(C=0.01, penalty="l1", dual=False))),
      #('normalization', StandardScaler()),
      ('classification', DecisionTreeClassifier())
    ])
    dt_clf = GridSearchCV(nb_pipe, nb_params, cv=4, n_jobs=-1, scoring='f1_macro')
    dt_clf.fit(new_X_train[LOCI], new_X_train[TARGET])
    
    print("!!!!!!dt!!!!!!!")
    report_acc(dt_clf.best_estimator_, new_X_train)
    
    rf_params = {
        'classification__max_leaf_nodes': [1, 3, 16, 32, 64, 200],
        'classification__class_weight': [None, 'balanced']
    }

    rf_pipe = Pipeline([
      #('feature_selection', SelectFromModel(LinearSVC(C=0.01, penalty="l1", dual=False))),
      #('normalization', StandardScaler()),
      ('classification', RandomForestClassifier())
    ])
    rf_clf = GridSearchCV(nb_pipe, nb_params, cv=3, n_jobs=-1, scoring='f1_macro')
    rf_clf.fit(new_X_train[LOCI], new_X_train[TARGET])
    
    print("!!!!!!RF!!!!!!!")
    report_acc(rf_clf.best_estimator_, new_X_train)
    
    svc_params = {
        'classification__penalty': ['l1', 'l2'],
        'classification__tol': [1e-4],
        'classification__C': [1e-1, 1e-2, 1e1, 1, 1e2],
    }

    svc_pipe = Pipeline([
      #('feature_selection', SelectFromModel(LinearSVC(C=0.01, penalty="l1", dual=False))),
      #('normalization', StandardScaler()),
      ('classification', LinearSVC())
    ])
    svc_clf = GridSearchCV(svc_pipe, svc_params, cv=3, n_jobs=-1, scoring='f1_macro')
    svc_clf.fit(new_X_train[LOCI], new_X_train[TARGET])  
    print("!!!!!!SVC!!!!!!!")
    report_acc(svc_clf.best_estimator_, new_X_train)
    

check_oversampling(X_resampled)

[1]	valid_0's multi_logloss: 1.79249
[2]	valid_0's multi_logloss: 1.80107
[3]	valid_0's multi_logloss: 1.81016
[4]	valid_0's multi_logloss: 1.82525
[5]	valid_0's multi_logloss: 1.82965
[6]	valid_0's multi_logloss: 1.84615
[7]	valid_0's multi_logloss: 1.84576
[8]	valid_0's multi_logloss: 1.8509
[9]	valid_0's multi_logloss: 1.8578
[10]	valid_0's multi_logloss: 1.86094
[11]	valid_0's multi_logloss: 1.85015
[12]	valid_0's multi_logloss: 1.84868
[13]	valid_0's multi_logloss: 1.85314
[14]	valid_0's multi_logloss: 1.852
[15]	valid_0's multi_logloss: 1.85816
[16]	valid_0's multi_logloss: 1.86044
[17]	valid_0's multi_logloss: 1.86254
[18]	valid_0's multi_logloss: 1.86567
[19]	valid_0's multi_logloss: 1.87653
[20]	valid_0's multi_logloss: 1.87761
[21]	valid_0's multi_logloss: 1.88846
[22]	valid_0's multi_logloss: 1.89702
[23]	valid_0's multi_logloss: 1.89774
[24]	valid_0's multi_logloss: 1.90378
[25]	valid_0's multi_logloss: 1.90091
[26]	valid_0's multi_logloss: 1.90672
[27]	valid_0's multi_logl

/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


!!!!!!NN!!!!!!!
TRAIN: 
 ACC: 
 1.0 
                precision    recall  f1-score   support

Вост. Полесье       1.00      1.00      1.00       121
 Зап. Полесье       1.00      1.00      1.00       121
  Поднепровье       1.00      1.00      1.00       121
    Понеманье       1.00      1.00      1.00       121
     Поозерье       1.00      1.00      1.00       121
  Центральный       1.00      1.00      1.00       121

     accuracy                           1.00       726
    macro avg       1.00      1.00      1.00       726
 weighted avg       1.00      1.00      1.00       726

VALID: 
 ACC: 
 0.25 
                precision    recall  f1-score   support

Вост. Полесье       0.14      0.17      0.15         6
 Зап. Полесье       0.20      0.12      0.15         8
  Поднепровье       0.27      0.27      0.27        11
    Понеманье       0.20      0.22      0.21         9
     Поозерье       0.00      0.00      0.00         7
  Центральный       0.36      0.42      0.39        19


/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


**SMOTE**

In [168]:
from imblearn.over_sampling import SMOTE, ADASYN

smote_ = SMOTE(random_state=0)
X_resampled, y_resampled = smote_.fit_resample(X_train[LOCI].astype(str), X_train[TARGET])
X_resampled = X_resampled.astype(float)
X_resampled[TARGET] = y_resampled

In [169]:
check_oversampling(X_resampled)

[1]	valid_0's multi_logloss: 1.77878
[2]	valid_0's multi_logloss: 1.77864
[3]	valid_0's multi_logloss: 1.77947
[4]	valid_0's multi_logloss: 1.77369
[5]	valid_0's multi_logloss: 1.78132
[6]	valid_0's multi_logloss: 1.78636
[7]	valid_0's multi_logloss: 1.80108
[8]	valid_0's multi_logloss: 1.8039
[9]	valid_0's multi_logloss: 1.81382
[10]	valid_0's multi_logloss: 1.80925
[11]	valid_0's multi_logloss: 1.81264
[12]	valid_0's multi_logloss: 1.80883
[13]	valid_0's multi_logloss: 1.80262
[14]	valid_0's multi_logloss: 1.80748
[15]	valid_0's multi_logloss: 1.80336
[16]	valid_0's multi_logloss: 1.79676
[17]	valid_0's multi_logloss: 1.78978
[18]	valid_0's multi_logloss: 1.79251
[19]	valid_0's multi_logloss: 1.80285
[20]	valid_0's multi_logloss: 1.80534
[21]	valid_0's multi_logloss: 1.81653
[22]	valid_0's multi_logloss: 1.81589
[23]	valid_0's multi_logloss: 1.81949
[24]	valid_0's multi_logloss: 1.82822
[25]	valid_0's multi_logloss: 1.83803
[26]	valid_0's multi_logloss: 1.84155
[27]	valid_0's multi_l

/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


!!!!!!NN!!!!!!!
TRAIN: 
 ACC: 
 1.0 
                precision    recall  f1-score   support

Вост. Полесье       1.00      1.00      1.00       121
 Зап. Полесье       1.00      1.00      1.00       121
  Поднепровье       1.00      1.00      1.00       121
    Понеманье       1.00      1.00      1.00       121
     Поозерье       1.00      1.00      1.00       121
  Центральный       1.00      1.00      1.00       121

     accuracy                           1.00       726
    macro avg       1.00      1.00      1.00       726
 weighted avg       1.00      1.00      1.00       726

VALID: 
 ACC: 
 0.11666666666666667 
                precision    recall  f1-score   support

Вост. Полесье       0.00      0.00      0.00         6
 Зап. Полесье       0.20      0.12      0.15         8
  Поднепровье       0.00      0.00      0.00        11
    Понеманье       0.06      0.11      0.08         9
     Поозерье       0.08      0.14      0.11         7
  Центральный       0.27      0.21      

/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


!!!!!!RF!!!!!!!
TRAIN: 
 ACC: 
 0.3856749311294766 
                precision    recall  f1-score   support

Вост. Полесье       0.54      0.31      0.39       121
 Зап. Полесье       0.40      0.61      0.49       121
  Поднепровье       0.31      0.44      0.36       121
    Понеманье       0.38      0.50      0.43       121
     Поозерье       0.40      0.28      0.33       121
  Центральный       0.38      0.18      0.25       121

     accuracy                           0.39       726
    macro avg       0.40      0.39      0.37       726
 weighted avg       0.40      0.39      0.37       726

VALID: 
 ACC: 
 0.13333333333333333 
                precision    recall  f1-score   support

Вост. Полесье       0.00      0.00      0.00         6
 Зап. Полесье       0.07      0.12      0.09         8
  Поднепровье       0.10      0.09      0.10        11
    Понеманье       0.17      0.33      0.22         9
     Поозерье       0.33      0.14      0.20         7
  Центральный       0.29 

/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


# Under sampling

In [45]:
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=0)
X_resampled, y_resampled = cc.fit_resample(X_train[LOCI], X_train[TARGET])
X_resampled[TARGET] = y_resampled

In [46]:
check_oversampling(X_resampled)

[1]	valid_0's multi_logloss: 1.79387
[2]	valid_0's multi_logloss: 1.79721
[3]	valid_0's multi_logloss: 1.80619
[4]	valid_0's multi_logloss: 1.81339
[5]	valid_0's multi_logloss: 1.82304
[6]	valid_0's multi_logloss: 1.8296
[7]	valid_0's multi_logloss: 1.84202
[8]	valid_0's multi_logloss: 1.84737
[9]	valid_0's multi_logloss: 1.8662
[10]	valid_0's multi_logloss: 1.87152
[11]	valid_0's multi_logloss: 1.87722
[12]	valid_0's multi_logloss: 1.88323
[13]	valid_0's multi_logloss: 1.89757
[14]	valid_0's multi_logloss: 1.90876
[15]	valid_0's multi_logloss: 1.90759
[16]	valid_0's multi_logloss: 1.91877
[17]	valid_0's multi_logloss: 1.93069
[18]	valid_0's multi_logloss: 1.93572
[19]	valid_0's multi_logloss: 1.94806
[20]	valid_0's multi_logloss: 1.95287
[21]	valid_0's multi_logloss: 1.96163
[22]	valid_0's multi_logloss: 1.97027
[23]	valid_0's multi_logloss: 1.96926
[24]	valid_0's multi_logloss: 1.97365
[25]	valid_0's multi_logloss: 1.98563
[26]	valid_0's multi_logloss: 1.99281
[27]	valid_0's multi_lo

/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


!!!!!!NN!!!!!!!
TRAIN: 
 ACC: 
 0.43162393162393164 
                precision    recall  f1-score   support

Вост. Полесье       0.37      0.62      0.46        39
 Зап. Полесье       0.44      0.38      0.41        39
  Поднепровье       0.50      0.44      0.47        39
    Понеманье       0.51      0.46      0.49        39
     Поозерье       0.32      0.18      0.23        39
  Центральный       0.45      0.51      0.48        39

     accuracy                           0.43       234
    macro avg       0.43      0.43      0.42       234
 weighted avg       0.43      0.43      0.42       234

VALID: 
 ACC: 
 0.2 
                precision    recall  f1-score   support

Вост. Полесье       0.06      0.17      0.09         6
 Зап. Полесье       0.21      0.38      0.27         8
  Поднепровье       0.30      0.27      0.29        11
    Понеманье       0.33      0.22      0.27         9
     Поозерье       0.00      0.00      0.00         7
  Центральный       0.27      0.16      

/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [47]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(X_train[LOCI], X_train[TARGET])
X_resampled[TARGET] = y_resampled

In [48]:
check_oversampling(X_resampled)

[1]	valid_0's multi_logloss: 1.80442
[2]	valid_0's multi_logloss: 1.81102
[3]	valid_0's multi_logloss: 1.81705
[4]	valid_0's multi_logloss: 1.82054
[5]	valid_0's multi_logloss: 1.83274
[6]	valid_0's multi_logloss: 1.83299
[7]	valid_0's multi_logloss: 1.84218
[8]	valid_0's multi_logloss: 1.85479
[9]	valid_0's multi_logloss: 1.85118
[10]	valid_0's multi_logloss: 1.85767
[11]	valid_0's multi_logloss: 1.86595
[12]	valid_0's multi_logloss: 1.86943
[13]	valid_0's multi_logloss: 1.87545
[14]	valid_0's multi_logloss: 1.88164
[15]	valid_0's multi_logloss: 1.89248
[16]	valid_0's multi_logloss: 1.89781
[17]	valid_0's multi_logloss: 1.90798
[18]	valid_0's multi_logloss: 1.90468
[19]	valid_0's multi_logloss: 1.91581
[20]	valid_0's multi_logloss: 1.92005
[21]	valid_0's multi_logloss: 1.92955
[22]	valid_0's multi_logloss: 1.93292
[23]	valid_0's multi_logloss: 1.93432
[24]	valid_0's multi_logloss: 1.94317
[25]	valid_0's multi_logloss: 1.95466
[26]	valid_0's multi_logloss: 1.96203
[27]	valid_0's multi_

/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


!!!!!!NN!!!!!!!
TRAIN: 
 ACC: 
 1.0 
                precision    recall  f1-score   support

Вост. Полесье       1.00      1.00      1.00        39
 Зап. Полесье       1.00      1.00      1.00        39
  Поднепровье       1.00      1.00      1.00        39
    Понеманье       1.00      1.00      1.00        39
     Поозерье       1.00      1.00      1.00        39
  Центральный       1.00      1.00      1.00        39

     accuracy                           1.00       234
    macro avg       1.00      1.00      1.00       234
 weighted avg       1.00      1.00      1.00       234

VALID: 
 ACC: 
 0.11666666666666667 
                precision    recall  f1-score   support

Вост. Полесье       0.00      0.00      0.00         6
 Зап. Полесье       0.17      0.12      0.14         8
  Поднепровье       0.00      0.00      0.00        11
    Понеманье       0.33      0.22      0.27         9
     Поозерье       0.00      0.00      0.00         7
  Центральный       0.25      0.21      

/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


!!!!!!RF!!!!!!!
TRAIN: 
 ACC: 
 0.358974358974359 
                precision    recall  f1-score   support

Вост. Полесье       0.29      0.23      0.26        39
 Зап. Полесье       0.30      0.49      0.37        39
  Поднепровье       0.33      0.44      0.38        39
    Понеманье       0.41      0.31      0.35        39
     Поозерье       0.45      0.26      0.33        39
  Центральный       0.45      0.44      0.44        39

     accuracy                           0.36       234
    macro avg       0.37      0.36      0.35       234
 weighted avg       0.37      0.36      0.35       234

VALID: 
 ACC: 
 0.18333333333333332 
                precision    recall  f1-score   support

Вост. Полесье       0.00      0.00      0.00         6
 Зап. Полесье       0.14      0.38      0.20         8
  Поднепровье       0.08      0.09      0.09        11
    Понеманье       0.25      0.22      0.24         9
     Поозерье       0.14      0.14      0.14         7
  Центральный       0.57  

/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


# Custom over sampling

In [161]:
from collections import defaultdict
import tqdm

def get_pij(X, y, locus):
    result = defaultdict(dict)
    alleles = X[locus].unique()
    POPULATIONS = X.Ethno_hist_region.unique()
    for population in POPULATIONS:
        for allele in alleles:
            a = (X[locus] == allele) & (y == population)
            b = (y == population)
            result[population][allele] = np.sum(a) / np.sum(b)
    return result

def get_pj(X, y, locus):
    pij = get_pij(X, y, locus)
    alleles = X[locus].unique()
    result = dict()
    for j in alleles:
        result[j] = np.mean([pij[i][j] for i in pij.keys() if j in pij[i]])
    return result

def custom_over_sampling(train_df, kk=1):
    result_df = []
    for pop, sub_df in train_df.groupby(TARGET):
        aa = np.random.choice(len(sub_df), kk, True)
        base_df = sub_df.reset_index(drop=True).iloc[aa]
        for index, row in tqdm.tqdm_notebook(base_df.iterrows()):
            locus = np.random.choice(len(LOCI), 1)[0]
            pij = get_pij(train_df, train_df[TARGET], LOCI[locus])
            k = list(pij[pop].keys())
            v = list(pij[pop].values())
            new_allele = k[np.random.choice(len(k), 1, p = v)[0]]
            #print("\n\n\nOLD ", base_df.loc[index, LOCI[locus]])
            base_df.loc[index, LOCI[locus]] = new_allele
            #print("\n\n\nNew ", base_df.loc[index, LOCI[locus]])

        result_df.append(base_df)
    return pd.concat(result_df, axis=0).reset_index(drop=True)

In [179]:
new_train_X = custom_over_sampling(X_train, 200)

/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/ipykernel_launcher.py:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [180]:
check_oversampling(new_train_X)

[1]	valid_0's multi_logloss: 1.80067
[2]	valid_0's multi_logloss: 1.7982
[3]	valid_0's multi_logloss: 1.82343
[4]	valid_0's multi_logloss: 1.83155
[5]	valid_0's multi_logloss: 1.8396
[6]	valid_0's multi_logloss: 1.85057
[7]	valid_0's multi_logloss: 1.84935
[8]	valid_0's multi_logloss: 1.86335
[9]	valid_0's multi_logloss: 1.86828
[10]	valid_0's multi_logloss: 1.86829
[11]	valid_0's multi_logloss: 1.87415
[12]	valid_0's multi_logloss: 1.878
[13]	valid_0's multi_logloss: 1.88162
[14]	valid_0's multi_logloss: 1.8925
[15]	valid_0's multi_logloss: 1.8986
[16]	valid_0's multi_logloss: 1.90532
[17]	valid_0's multi_logloss: 1.90229
[18]	valid_0's multi_logloss: 1.90262
[19]	valid_0's multi_logloss: 1.90752
[20]	valid_0's multi_logloss: 1.90804
[21]	valid_0's multi_logloss: 1.90597
[22]	valid_0's multi_logloss: 1.91058
[23]	valid_0's multi_logloss: 1.91699
[24]	valid_0's multi_logloss: 1.92436
[25]	valid_0's multi_logloss: 1.9298
[26]	valid_0's multi_logloss: 1.93625
[27]	valid_0's multi_logloss

/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


!!!!!!NN!!!!!!!
TRAIN: 
 ACC: 
 1.0 
                precision    recall  f1-score   support

Вост. Полесье       1.00      1.00      1.00       200
 Зап. Полесье       1.00      1.00      1.00       200
  Поднепровье       1.00      1.00      1.00       200
    Понеманье       1.00      1.00      1.00       200
     Поозерье       1.00      1.00      1.00       200
  Центральный       1.00      1.00      1.00       200

     accuracy                           1.00      1200
    macro avg       1.00      1.00      1.00      1200
 weighted avg       1.00      1.00      1.00      1200

VALID: 
 ACC: 
 0.18333333333333332 
                precision    recall  f1-score   support

Вост. Полесье       0.12      0.17      0.14         6
 Зап. Полесье       0.25      0.25      0.25         8
  Поднепровье       0.00      0.00      0.00        11
    Понеманье       0.00      0.00      0.00         9
     Поозерье       0.12      0.14      0.13         7
  Центральный       0.37      0.37      

/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


!!!!!!NB!!!!!!!
TRAIN: 
 ACC: 
 0.3925 
                precision    recall  f1-score   support

Вост. Полесье       0.54      0.54      0.54       200
 Зап. Полесье       0.34      0.53      0.41       200
  Поднепровье       0.27      0.21      0.23       200
    Понеманье       0.43      0.48      0.46       200
     Поозерье       0.43      0.29      0.35       200
  Центральный       0.36      0.30      0.33       200

     accuracy                           0.39      1200
    macro avg       0.39      0.39      0.39      1200
 weighted avg       0.39      0.39      0.39      1200

VALID: 
 ACC: 
 0.21666666666666667 
                precision    recall  f1-score   support

Вост. Полесье       0.14      0.33      0.20         6
 Зап. Полесье       0.14      0.25      0.18         8
  Поднепровье       0.17      0.09      0.12        11
    Понеманье       0.27      0.44      0.33         9
     Поозерье       0.50      0.29      0.36         7
  Центральный       0.29      0.11   

/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


!!!!!!NB!!!!!!!
TRAIN: 
 ACC: 
 0.3925 
                precision    recall  f1-score   support

Вост. Полесье       0.54      0.54      0.54       200
 Зап. Полесье       0.34      0.53      0.41       200
  Поднепровье       0.27      0.21      0.23       200
    Понеманье       0.43      0.48      0.46       200
     Поозерье       0.43      0.29      0.35       200
  Центральный       0.36      0.30      0.33       200

     accuracy                           0.39      1200
    macro avg       0.39      0.39      0.39      1200
 weighted avg       0.39      0.39      0.39      1200

VALID: 
 ACC: 
 0.21666666666666667 
                precision    recall  f1-score   support

Вост. Полесье       0.14      0.33      0.20         6
 Зап. Полесье       0.14      0.25      0.18         8
  Поднепровье       0.17      0.09      0.12        11
    Понеманье       0.27      0.44      0.33         9
     Поозерье       0.50      0.29      0.36         7
  Центральный       0.29      0.11   

/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


!!!!!!RF!!!!!!!
TRAIN: 
 ACC: 
 0.3925 
                precision    recall  f1-score   support

Вост. Полесье       0.54      0.54      0.54       200
 Зап. Полесье       0.34      0.53      0.41       200
  Поднепровье       0.27      0.21      0.23       200
    Понеманье       0.43      0.48      0.46       200
     Поозерье       0.43      0.29      0.35       200
  Центральный       0.36      0.30      0.33       200

     accuracy                           0.39      1200
    macro avg       0.39      0.39      0.39      1200
 weighted avg       0.39      0.39      0.39      1200

VALID: 
 ACC: 
 0.21666666666666667 
                precision    recall  f1-score   support

Вост. Полесье       0.14      0.33      0.20         6
 Зап. Полесье       0.14      0.25      0.18         8
  Поднепровье       0.17      0.09      0.12        11
    Понеманье       0.27      0.44      0.33         9
     Поозерье       0.50      0.29      0.36         7
  Центральный       0.29      0.11   

/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [183]:
y_pred = rf_clf.best_estimator_.predict(test_df[LOCI])
y_true = test_df[TARGET]
print("TRAIN: \n", "ACC: \n", np.mean(y_pred == y_true), "\n", classification_report(y_true, y_pred))
    

TRAIN: 
 ACC: 
 0.27 
                precision    recall  f1-score   support

Вост. Полесье       0.00      0.00      0.00        10
 Зап. Полесье       0.00      0.00      0.00        13
  Поднепровье       0.17      0.05      0.08        19
    Понеманье       0.00      0.00      0.00        15
     Поозерье       0.00      0.00      0.00        12
  Центральный       0.30      0.84      0.44        31

     accuracy                           0.27       100
    macro avg       0.08      0.15      0.09       100
 weighted avg       0.12      0.27      0.15       100



/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [176]:
report_acc(rf_clf.best_estimator_, train_df)

TRAIN: 
 ACC: 
 0.328125 
                precision    recall  f1-score   support

Вост. Полесье       0.00      0.00      0.00        45
 Зап. Полесье       0.34      0.20      0.26        59
  Поднепровье       0.43      0.03      0.06        87
    Понеманье       0.00      0.00      0.00        66
     Поозерье       0.00      0.00      0.00        51
  Центральный       0.33      0.94      0.48       140

     accuracy                           0.33       448
    macro avg       0.18      0.20      0.13       448
 weighted avg       0.23      0.33      0.20       448

VALID: 
 ACC: 
 0.2833333333333333 
                precision    recall  f1-score   support

Вост. Полесье       0.00      0.00      0.00         6
 Зап. Полесье       0.14      0.12      0.13         8
  Поднепровье       0.00      0.00      0.00        11
    Понеманье       0.00      0.00      0.00         9
     Поозерье       0.00      0.00      0.00         7
  Центральный       0.30      0.84      0.44        

/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/dmitriydemidko/anaconda3/envs/env37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
